In [0]:
location = "abfss://<container-name>@<storage-account>.dfs.core.windows.net/Brazil_ECommerce/"

In [0]:
df = spark.read.option("mode", "DROPMALFORMED").option("header", True).option("infer_Schema", True).csv(location+"Curated/brazil_e_commerce_curated.csv")

In [0]:
from pyspark.sql.functions import col,lit,when

df_products = df.select("Product_Id", "Product_Category", "Price", "Freight_Value").distinct()

df_products.coalesce(1).write.format("csv").save(location + "Developed/Temp",header = True)

filenames = dbutils.fs.ls(location + "Developed/Temp")
name = ''

for filename in filenames:
    if filename.name.endswith('.csv'):
        name = filename.name

dbutils.fs.cp(location + "Developed/Temp/" + name, location + "Developed/Brazil_ECommerce_Products.csv")
dbutils.fs.rm(location + "Developed/Temp/",recurse = True)

Out[4]: True

In [0]:
df_customers = df.select("Customer_Unique_Id", "Customer_Zip_Code", "Customer_City", "Customer_State").distinct()

df_customers.coalesce(1).write.format("csv").save(location + "Developed/Temp",header = True)

filenames = dbutils.fs.ls(location + "Developed/Temp")
name = ''

for filename in filenames:
    if filename.name.endswith('.csv'):
        name = filename.name

dbutils.fs.cp(location + "Developed/Temp/" + name, location + "Developed/Brazil_ECommerce_Customers.csv")
dbutils.fs.rm(location + "Developed/Temp/",recurse = True)

Out[5]: True

In [0]:
df_sellers = df.select("Seller_Id", "Seller_Zip_Code", "Seller_City", "Seller_State").distinct()

df_sellers.coalesce(1).write.format("csv").save(location + "Developed/Temp",header = True)

filenames = dbutils.fs.ls(location + "Developed/Temp")
name = ''

for filename in filenames:
    if filename.name.endswith('.csv'):
        name = filename.name

dbutils.fs.cp(location + "Developed/Temp/" + name, location + "Developed/Brazil_ECommerce_Sellers.csv")
dbutils.fs.rm(location + "Developed/Temp/",recurse = True)

Out[6]: True

In [0]:
df_reviews = df.select("Review_Id", "Review_Score").distinct()


df_reviews.coalesce(1).write.format("csv").save(location + "Developed/Temp",header = True)

filenames = dbutils.fs.ls(location + "Developed/Temp")
name = ''

for filename in filenames:
    if filename.name.endswith('.csv'):
        name = filename.name

dbutils.fs.cp(location + "Developed/Temp/" + name, location + "Developed/Brazil_ECommerce_Reviews.csv")
dbutils.fs.rm(location + "Developed/Temp/",recurse = True)

Out[7]: True

In [0]:
##When Required columns is more than Not required columns, it is better to apply drop
df_orders = df.select("Order_Id", "Order_Status", "Product_Id", "Order_Item_Quantity", "Customer_Unique_Id", "Seller_Id", "Total_Payment", "Review_Id", "Order_Purchased_Dates", "Order_Estimated_Delivery_Dates", "Order_Delivered_Customer_Dates")

df_orders.coalesce(1).write.format("csv").save(location + "Developed/Temp",header = True)

filenames = dbutils.fs.ls(location + "Developed/Temp")
name = ''

for filename in filenames:
    if filename.name.endswith('.csv'):
        name = filename.name

dbutils.fs.cp(location + "Developed/Temp/" + name, location + "Developed/Brazil_ECommerce_Orders.csv")
dbutils.fs.rm(location + "Developed/Temp/",recurse = True)

Out[10]: True

In [0]:
from pyspark.sql.functions import datediff

df_orders_analysed = df_orders.select("Order_Id","Customer_Unique_Id","Seller_Id","Review_Id",
                                     (datediff(col("Order_Estimated_Delivery_Dates"), col("Order_Delivered_Customer_Dates")).alias("Order_Delivery_Delay_Days")),
                                         (datediff(col("Order_Delivered_Customer_Dates"), col("Order_Purchased_Dates"))).alias("Order_Life_Days"))

In [0]:
df_orders_analysed = df_orders_analysed.withColumn("Delay_in_Delivery", lit(when(col("Order_Delivery_Delay_Days")<0, "Yes").otherwise("No")))

In [0]:
df_orders_analysed = df_orders_analysed.select("*").join(df_reviews, df_orders_analysed["Review_Id"]==df_reviews["Review_Id"],"inner").drop("Review_Id").distinct()

In [0]:
df_orders_analysed.coalesce(1).write.format("csv").save(location + "Developed/Temp",header = True)

filenames = dbutils.fs.ls(location + "Developed/Temp")
name = ''

for filename in filenames:
    if filename.name.endswith('.csv'):
        name = filename.name

dbutils.fs.cp(location + "Developed/Temp/" + name, location + "Developed/Brazil_ECommerce_Orders_Agg.csv")
dbutils.fs.rm(location + "Developed/Temp/",recurse = True)

Out[15]: True